In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold



In [2]:
df2 = pd.DataFrame()
n = [3, 4, 5, 12, 13, 14, 23, 24, 25, 44]
for i in range(1, 76):
    if i not in n:
        df = pd.read_excel("B:\GroundwaterRech\GithubRepo\Restart\Dataset\District Wise\\" + str(i) + ".xlsx")
        df1 = df.drop(['Year', 'Longitude', 'Latitude'], axis=1)
        df2 = pd.concat([df2, df1])

In [3]:
df2['Soil Type'] = df2['Soil Type'].replace('Builtup', 1)
df2['Soil Type'] = df2['Soil Type'].replace('Bulitup', 1)

df2['Soil Type'] = df2['Soil Type'].replace('Haplic Luvisols', 2)
df2['Soil Type'] = df2['Soil Type'].replace('Haplic Calcisols', 3)
df2['Soil Type'] = df2['Soil Type'].replace('Loam', 4)
df2['Soil Type'] = df2['Soil Type'].replace('Calcaric Fluvisols', 5)


In [4]:
df2

,District,Slope,Soil Type,LULC,Max Temperature,Min Temperature,Precipitation,Groundwater Recharge
0,Faridkot,0.496398,1,Agriculture,42.902,21.724,74.26,-0.09
1,Faridkot,0.496398,1,Agriculture,45.352,23.562,39.16,0.83
2,Faridkot,0.496398,1,Agriculture,41.710,22.326,120.64,-1.48
3,Faridkot,0.496398,1,Agriculture,43.788,22.494,79.64,0.30
4,Faridkot,0.496398,1,Agriculture,42.016,22.742,105.90,-0.80
...,...,...,...,...,...,...,...,...
29,Shahid Bhagat Singh Nagar,2.827676,5,Builtup,41.738,21.140,216.40,0.50
30,Shahid Bhagat Singh Nagar,2.827676,5,Builtup,42.502,22.722,210.86,0.30
31,Shahid Bhagat Singh Nagar,2.827676,5,Builtup,42.956,21.792,233.44,0.65
32,Shahid Bhagat Singh Nagar,2.827676,5,Builtup,42.238,22.246,246.28,0.10


In [5]:
X = df2.drop('Groundwater Recharge', axis=1)
y = df2['Groundwater Recharge']

In [6]:
X = pd.get_dummies(X)

In [7]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = MinMaxScaler()
# train_data = scaler.fit_transform(train_data)
# test_data =scaler.fit_transform(test_data)

`PCA`

In [10]:
from sklearn.decomposition import PCA

In [11]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(train_data)
X_test_pca = pca.fit_transform(test_data)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [13]:
d = {}

In [14]:
def get_model_results(model, n, d):
    model.fit(X_train_pca, train_labels)
    y_pred = model.predict(X_test_pca)
    y_pred_train = model.predict(X_train_pca)
    r2 = r2_score(test_labels, y_pred)
    ms = mean_squared_error(test_labels, y_pred)
    print(f'R2_score: {r2} RMSE: {math.sqrt(ms)}')

    train_score = mean_squared_error(train_labels, y_pred_train)
    test_score = mean_squared_error(test_labels, y_pred)
    r2_s = r2_score(test_labels, y_pred)
    mae_test = mean_absolute_error(test_labels, y_pred)
    mae_train = mean_absolute_error(train_labels, y_pred_train)

    d[n] = (train_score, test_score, r2_s, mae_train, mae_test)

In [15]:
model = LinearRegression()
get_model_results(model, 'Linear Regression', d)

R2_score: 0.018026726922892977 RMSE: 2.2970722784107087


In [17]:
d

{'Linear Regression': (3.7093451931294186,
  5.276517794035717,
  0.01803105531485838,
  1.0599373228827165,
  1.2703383763309635)}

In [18]:
model = RandomForestRegressor(max_depth=12)
get_model_results(model, 'Random Forest', d)

R2_score: -0.2133007572590193 RMSE: 2.5533424247026595


In [115]:
d

{'Linear Regression': (3.4835870220721583,
  5.367305802168324,
  0.0011352524357677307,
  1.0383647847534805,
  1.3150781755518004),
 'Random Forest': (0.6223010158345319,
  5.714927758175672,
  -0.06355778539986101,
  0.5512811627612845,
  1.338251897168827)}